In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install praw
!pip install transformers torch
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 206.1 kB/s eta 0:00:00 0:00:01


In [5]:
from datasets import load_dataset

ds = load_dataset("ccdv/arxiv-summarization", "document")

In [6]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
import torch

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Preprocess Function
def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["abstract"], max_length=128, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Apply preprocessing on only 10,000 samples
ds_small = ds["train"].select(range(10000))  # Select first 10k samples
tokenized_ds = ds_small.map(preprocess_function, batched=True)

Using device: cuda


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [7]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)
# Training Arguments (removed 'device' argument)
training_args = TrainingArguments(
    output_dir="./fine-tuned-model",
    eval_strategy="epoch",               # Updated here
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",                   # Disable W&B
)


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,  # Use the 10k subset
    eval_dataset=tokenized_ds,   # Optional: same for validation if separate validation isn't available
)

# Fine-tune the model
trainer.train()

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,2.291700,2.010976
2,1.974400,1.797372
3,1.846800,1.703413


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=7500, training_loss=2.1029660046895344, metrics={'train_runtime': 11542.4153, 'train_samples_per_second': 2.599, 'train_steps_per_second': 0.65, 'total_flos': 6.501313806336e+16, 'train_loss': 2.1029660046895344, 'epoch': 3.0})

In [ ]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

model.save_pretrained("saved_model")
tokenizer_ds.save_pretrained("saved_model")

# Load the saved model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("saved_model")
tokenizer = AutoTokenizer.from_pretrained("saved_model")

# Initialize the summarizer pipeline with the loaded model and tokenizer
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)

In [ ]:
# Replace with your Reddit API credentials
CLIENT_ID = "G6R7wy_ArTl2L3zuqFB8sA"
CLIENT_SECRET = "H1Hv2Ys46-wkgJmD4B6_gL-3-ACGHw"
USER_AGENT = "Flimsy_Mouse_622"

# Authenticate
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)


# Fetch the latest posts from a subreddit
def fetch_posts(subreddit_name, limit=5):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    for post in subreddit.new(limit=limit):
        posts.append({
            "title": post.title,
            "body": post.selftext,
            "url": post.url
        })
    return posts

import re

def clean_text(text):
    """
    Clean the text by removing special characters, URLs, and extra spaces.
    """
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    # Remove special characters (keep only alphanumeric and basic punctuation)
    text = re.sub(r"[^a-zA-Z0-9\s.,!?]", "", text)
    # Remove extra spaces
    text = " ".join(text.split())
    return text

def summarize_post(post):
    """
    Summarize a post's title and body.
    """
    text = f"{post['title']}. {post['body']}"
    text = clean_text(text)  # Clean the text
    print(f"Text to summarize: {text}")  # Debug print

    # Skip summarization if the text is too short or too long
    if len(text.split()) < 5:  # Skip if the text is too short
        print("Skipping summarization: Text is too short.")
        return post["title"]  # Return the title as a fallback
    if len(text.split()) > 1024:  # Truncate if the text is too long
        print("Truncating text: Text is too long.")
        text = " ".join(text.split()[:1024])

    # Summarize the text
    try:
        summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return post["title"]  # Return the title as a fallback

# Summarize multiple posts
def summarize_posts(posts):
    """
    Summarize a list of posts.
    """
    summaries = []
    for post in posts:
        summary = summarize_post(post)
        summaries.append({
            "title": post["title"],
            "summary": summary,
            "url": post["url"]
        })
    return summaries

# Example usage
subreddit_name = "MachineLearning"
posts = fetch_posts(subreddit_name, limit=5)
summaries = summarize_posts(posts)

# Print the results
for summary in summaries:
    print("Title:", summary["title"])
    print("Summary:", summary["summary"])
    print("URL:", summary["url"])
    print("---")